In [1]:
# import zipfile as zf
# Cars_DS_Path ='Datasets/Cars_Dataset_v2.zip'
# with zf.ZipFile(Cars_DS_Path, 'r') as zip_ref:
#     zip_ref.extractall('Datasets/Cars_Dataset_v2')

In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


2023-07-27 00:20:29.260258: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 00:20:29.323505: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 00:20:30.463055: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Specify the path to the dataset directory
dataset_dir = 'Datasets/Tumor_detection/Training'


image_height = 100
image_width = 100

target_size = (image_height, image_width)

# Initialize lists to store the images and labels
images = []
labels = []

# Iterate over the subdirectories in the dataset directory
for subdir in os.listdir(dataset_dir):
    subdir_path = os.path.join(dataset_dir, subdir)
    if os.path.isdir(subdir_path):
        # Extract the class label from the subdirectory name
        label = subdir
        # Iterate over the image files in the subdirectory
        for file_name in os.listdir(subdir_path):
            # Read the image file
            image_path = os.path.join(subdir_path, file_name)
            image = cv2.imread(image_path)
            # Preprocess the image (resize, normalize, etc.)
            image = cv2.resize(image, target_size)
            # Add the preprocessed image and label to the lists
            images.append(image)
            labels.append(label)

# Convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels.shape)

(5712, 100, 100, 3)
(5712,)


In [4]:
# Perform any required preprocessing steps on the images and labels

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Split the dataset into training and testing sets
train_images, test_images, train_labels0, test_labels0 = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convert the labels to one-hot encoded vectors
num_classes = len(np.unique(labels))

print(num_classes)

label_encoder = LabelEncoder()
train_labels1 = label_encoder.fit_transform(train_labels0)
test_labels1  = label_encoder.fit_transform(test_labels0)


from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels1, num_classes)
test_labels = to_categorical(test_labels1, num_classes)


4


In [5]:
model = Sequential()



# Add convolutional and pooling layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add more convolutional and pooling layers if desired

# Flatten the output from the previous layer
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])






2023-07-27 00:20:45.228638: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-27 00:20:45.254989: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
num_epochs = 1
batch_size = 10


print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(4569, 100, 100, 3)
(4569, 4)
(1143, 100, 100, 3)
(1143, 4)


In [7]:

model.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size, validation_data=(test_images, test_labels))




322/457 [====================>.........] - ETA: 20s - loss: 57.2437 - accuracy: 0.6860

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

36/36 [==============================] - 2s 62ms/step - loss: 0.7148 - accuracy: 0.8801
Test Loss: 0.7148
Test Accuracy: 0.8801


In [ ]:
# # Perform any necessary preprocessing steps on the input image

# # Make predictions on the preprocessed image
# prediction = model.predict(test_images)

# # Process the prediction (e.g., retrieve the predicted class label)
# predicted_label = np.argmax(prediction)
# print(predicted_label)